# Imports and configs

In [1]:
!pip install -q autogluon.tabular ray==2.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 98.0 MB/s eta 0:00:00


In [2]:
from autogluon.tabular import TabularPredictor
import pandas as pd
import numpy as np
import shutil

In [3]:
class CFG:
    train_path = '/kaggle/input/ai-competition-africa/train.csv'
    test_path = '/kaggle/input/ai-competition-africa/test.csv'
    sample_sub_path = '/kaggle/input/ai-competition-africa/sample_submission.csv'
    original_data_path = '/kaggle/input/wine-quality-dataset/WineQT.csv'
    
    target = 'quality'
    seed = 1807
    time_limit = 3600 * 1

# Loading data and predefining folds

In [4]:
train = pd.read_csv(CFG.train_path, index_col='id')
test = pd.read_csv(CFG.test_path, index_col='id')
original = pd.read_csv(CFG.original_data_path, index_col='Id')

train[CFG.target] = train[CFG.target].astype(int)
original[CFG.target] = original[CFG.target].astype(int)

mapping = {3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5}
train[CFG.target] = train[CFG.target].map(mapping)
original[CFG.target] = original[CFG.target].map(mapping)

# Fitting the predictor

In [5]:
predictor = TabularPredictor(
    problem_type='multiclass',
    eval_metric='quadratic_kappa',
    label=CFG.target,
    verbosity=2
)

No path specified. Models will be saved in: "AutogluonModels/ag-20241222_212925"


In [6]:
predictor.fit_pseudolabel(
    train_data=train,
    pseudo_data=original,
    time_limit=CFG.time_limit,
    excluded_model_types = ["KNN"],
    presets='best_quality',
    ag_args_fit={
        'num_gpus': 1, 
        'num_cpus': 4
    }
)

Predictor not fit prior to pseudolabeling. Fitting now...
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.21 GB / 31.35 GB (96.4%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the dat

In [7]:
predictor.leaderboard(silent=True).style.background_gradient(subset=['score_val'], cmap='RdYlGn')

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3_PSEUDO,0.412030,quadratic_kappa,21.602748,4061.519519,0.002984,2.795523,3,True,72
1,WeightedEnsemble_L3,0.411244,quadratic_kappa,9.380044,1853.399850,0.003050,2.136887,3,True,32
2,NeuralNetTorch_BAG_L2,0.410226,quadratic_kappa,9.376993,1851.262963,0.608973,94.106611,2,True,30
3,NeuralNetTorch_BAG_L2_PSEUDO,0.410104,quadratic_kappa,12.222770,2207.461033,0.789894,93.661742,2,True,69
4,WeightedEnsemble_L2,0.402039,quadratic_kappa,1.192183,418.370192,0.003002,1.612945,2,True,20
5,NeuralNetTorch_r22_BAG_L1,0.396578,quadratic_kappa,0.109341,119.597533,0.109341,119.597533,1,True,18
6,NeuralNetTorch_r79_BAG_L1,0.394949,quadratic_kappa,0.112630,140.119146,0.112630,140.119146,1,True,13
7,NeuralNetTorch_r22_BAG_L1_PSEUDO,0.394655,quadratic_kappa,0.108609,145.748338,0.108609,145.748338,1,True,50
8,WeightedEnsemble_L2_PSEUDO,0.394655,quadratic_kappa,0.112185,147.753989,0.003576,2.005651,2,True,59
9,NeuralNetTorch_r79_BAG_L1_PSEUDO,0.389971,quadratic_kappa,0.113227,137.487981,0.113227,137.487981,1,True,45


# Creating a submission file

In [8]:
test_pred_probs = predictor.predict_proba(test)
sub = pd.read_csv(CFG.sample_sub_path)
sub[CFG.target] = np.argmax(test_pred_probs, axis=1)
sub[CFG.target] = sub[CFG.target].map({0: 3, 1: 4, 2: 5, 3: 6, 4: 7, 5: 8})
sub.to_csv('sub_autogluon.csv', index=False)
sub.head()

,id,quality
0,15000,5
1,15001,5
2,15002,6
3,15003,6
4,15004,5


In [9]:
shutil.rmtree("AutogluonModels")